In [32]:
import pandas as pd

fake_df = pd.read_csv("Fake.csv")
print(fake_df.info())
fake_df = fake_df[:501]
print(fake_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    501 non-null    object
 1   text     501 non-null    object
 2   subject  501 non-null    object
 3   date     501 non-null    object
dtypes: object(4)
memory usage: 15.8+ KB
None


In [33]:
real_df = pd.read_csv("True.csv")
print(real_df.info())
real_df = real_df[:501]
print(real_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    501 non-null    object
 1   text     501 non-null    object
 2   subject  501 non-null    object
 3   date     501 non-null    object
dtypes: object(4)
memory usage: 15.8+ KB
None


In [34]:
real_df["target"]=1
fake_df["target"]=0

news = pd.concat([real_df,fake_df],axis=0)
print(news.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1002 entries, 0 to 500
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    1002 non-null   object
 1   text     1002 non-null   object
 2   subject  1002 non-null   object
 3   date     1002 non-null   object
 4   target   1002 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 47.0+ KB
None


In [35]:
X = news["title"] +" "+ news["text"]
y = news["target"]

print(X.head())

0    As U.S. budget fight looms, Republicans flip t...
1    U.S. military to accept transgender recruits o...
2    Senior U.S. Republican senator: 'Let Mr. Muell...
3    FBI Russia probe helped by Australian diplomat...
4    Trump wants Postal Service to charge 'much mor...
dtype: object


In [36]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aaron\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aaron\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [40]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()

def text_transform(data):
    item = re.sub('[^a-zA-Z]'," ",data)
    item = item.lower()
    item = item.split(" ")
    lema = []
    for word in item:
        if word not in stopwords.words("english"):
            lema.append(lemmatizer.lemmatize(word))
    return " ".join(lema)


In [41]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=8)


In [42]:
X_train = X_train.apply(text_transform)
print(X_train)

66     georgia  battle  staceys  test democrat  futur...
39     senate shelf disaster aid bill next month wash...
302    senator franken  facing resignation call  make...
475     ice may soon able destroy record immigrant de...
282    democratic lawmaker question kushner new york ...
                             ...                        
485     trump tried distance  senior advisor  promise...
133    republican finalized compromise u  tax bill  c...
361    u  corporate alternative minimum tax removed  ...
340    democrat cite drafting error proposed capital ...
451    uk pm may say donald trump wrong retweet far r...
Length: 801, dtype: object


In [43]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(ngram_range=(1,1))
features = v.fit_transform(X_train)



In [45]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(features,y_train)

RandomForestClassifier()

In [47]:
X_test = X_test.apply(text_transform)
X_test = v.transform(X_test)
pred = rfc.predict(X_test)

In [48]:
from sklearn.metrics import classification_report

error = classification_report(y_test,pred)
print(error)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       100
           1       1.00      1.00      1.00       101

    accuracy                           1.00       201
   macro avg       1.00      1.00      1.00       201
weighted avg       1.00      1.00      1.00       201



In [ ]:
news = input("Enter a news article!")
news = pd.Series(news)
news = news.apply(text_transform)
news = v.transform(news)
pred = rfc.predict(news)

if pred[0] == 1:
    print("This news item is real!")
else:
    print("This news item is fake!")

[1]
